In [1]:
%run ../ipynb_util_tars.py

In [9]:
sample_sentence = "Is this about poverty?"

In [10]:
"""load pretrained llama"""
import torch
from transformers import AutoTokenizer
from lxt.models.llama import LlamaForSequenceClassification, attnlrp

model_name = f"{CHECKPOINT_PATH}/meta-llama/Meta-Llama-3-8B-ft-zo_up/checkpoint-2200/"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LlamaForSequenceClassification.from_pretrained(
    model_name,
    num_labels=17,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
model.config.pad_token_id = tokenizer.pad_token_id

# apply AttnLRP rules
attnlrp.register(model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear(
            (base_layer): LinearEpsilon(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): LinearEpsilon(in_features=4096, out_features=1024, bias=False)
          (v_proj): lora.Linear(
            (base_layer): LinearEpsilon(in_features=4096, out_features=1024, bias=False)
  

In [11]:
# Tokenize the sentence
tokenized_output = tokenizer(sample_sentence, return_tensors="pt", add_special_tokens=True)

# Extract token IDs
token_ids = tokenized_output['input_ids'][0].tolist()

# Decode token IDs to readable string
decoded_sentence = tokenizer.decode(token_ids, skip_special_tokens=True)

print("Token IDs:", token_ids)
print("Decoded sentence:", decoded_sentence)

Token IDs: [128000, 3957, 420, 922, 19542, 30]
Decoded sentence: Is this about poverty?


In [12]:
from lxt.utils import clean_tokens

input_ids = tokenizer(
    sample_sentence,
    return_tensors="pt",
    add_special_tokens=True
).input_ids.to(model.device)
input_embeds = model.get_input_embeddings()(input_ids)

output_logits = model(inputs_embeds=input_embeds.requires_grad_(), use_cache=False).logits
print("output shape", output_logits.shape)

max_logits, max_indices = torch.max(output_logits[0, :], dim=-1)

max_logits.backward(max_logits)
relevance = input_embeds.grad.float().sum(-1).cpu()[0]

# normalize relevance between [-1, 1] for plotting
relevance = relevance / relevance.abs().max()

# remove '_' characters from token strings
tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
tokens = clean_tokens(tokens)

print(max_logits, max_indices.item())
print(tokenizer.decode(input_ids[0], skip_special_tokens=True))

output shape torch.Size([1, 17])
tensor(6.2500, device='cuda:0', dtype=torch.bfloat16, grad_fn=<MaxBackward0>) 12
Is this about poverty?


In [13]:
print(max_indices)

tensor(12, device='cuda:0')


In [14]:
print(tokens)
relevance

['<|begin\\_of\\_text|>', 'Is', 'Ġthis', 'Ġabout', 'Ġpoverty', '?']


tensor([ 1.0000, -0.3820, -0.0118,  0.0390, -0.0528,  0.1398])

# AttnLRP on LlamaForSequenceClassification

In [7]:
import torch
from transformers import AutoTokenizer
from lxt.models.llama import LlamaForSequenceClassification, attnlrp
from lxt.utils import pdf_heatmap, clean_tokens

model_name = "meta-llama/Meta-Llama-3-8B"
model = LlamaForSequenceClassification.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    num_labels=4,
    device_map="auto",
    token=HF_TOKEN
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# apply AttnLRP rules
attnlrp.register(model)

sample_sentence = "HAHAHAHAHAH This really just seems like a bunch of kino slop not rly a movie"

input_ids = tokenizer(
    sample_sentence,
    return_tensors="pt",
    add_special_tokens=True
).input_ids.to(model.device)
input_embeds = model.get_input_embeddings()(input_ids)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
output_logits = model(inputs_embeds=input_embeds.requires_grad_(), use_cache=False).logits
output_logits.shape

torch.Size([1, 4])

In [9]:
max_logits, max_indices = torch.max(output_logits[0, :], dim=-1)

max_logits.backward(max_logits)
relevance = input_embeds.grad.float().sum(-1).cpu()[0]

# normalize relevance between [-1, 1] for plotting
relevance = relevance / relevance.abs().max()

# remove '_' characters from token strings
tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
tokens = clean_tokens(tokens)

In [10]:
tokens

['<|begin\\_of\\_text|>',
 'HA',
 'HA',
 'HA',
 'HA',
 'HA',
 'H',
 'ĠThis',
 'Ġreally',
 'Ġjust',
 'Ġseems',
 'Ġlike',
 'Ġa',
 'Ġbunch',
 'Ġof',
 'Ġkino',
 'Ġslo',
 'p',
 'Ġnot',
 'Ġr',
 'ly',
 'Ġa',
 'Ġmovie']

In [11]:
relevance

tensor([ 1.0000,  0.2128,  0.0015, -0.0199, -0.0203, -0.0191,  0.0161,  0.0531,
         0.0041, -0.0271, -0.0949, -0.0189,  0.0204,  0.0020,  0.0094, -0.0442,
         0.0052, -0.0390,  0.2283, -0.0795, -0.0971, -0.0434,  0.6873])